# Research about H20 - 3 platform Machine Learning tools

# 1. Group information

| Student ID    | Name               | % contribution |
|---------------|--------------------|----------------|
| 20127681      | Nguyễn Thiên Phúc  | 100%           |
| 20127275      | Lê Nguyễn Nhật Phú | 100%           |
| 20127433      | Võ Hiền Hải Thuận  | 100%           |


## 2. H20 model demo

In [28]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.rulefit import H2ORuleFitEstimator
from h2o.estimators import H2OKMeansEstimator
from h2o.automl import H2OAutoML
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
h2o.init(ip="localhost", port=54321)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17.0.6+9-LTS-190, mixed mode, sharing)
  Starting server from C:\Users\Nguyen Thien Phuc\AppData\Local\Programs\Python\Python310\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\NGUYEN~1\AppData\Local\Temp\tmpspaqnm3j
  JVM stdout: C:\Users\NGUYEN~1\AppData\Local\Temp\tmpspaqnm3j\h2o_Nguyen_Thien_Phuc_started_from_python.out
  JVM stderr: C:\Users\NGUYEN~1\AppData\Local\Temp\tmpspaqnm3j\h2o_Nguyen_Thien_Phuc_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Ho_Chi_Minh
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_Nguyen_Thien_Phuc_ftqf5c
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.939 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


## 2.1 On small dataset

### a. Dataset description

**Name:** Prostate Dataset

| Variable | Definition                                            |
|----------|-------------------------------------------------------|
| AGE      |Tuổi của bệnh nhân                                     |
| RACE     |Chủng tộc của bệnh nhân                                |
| VOL      |Thể tích khối u(cm3)                                   |
| GLEASON  |Điểm số Gleason của bệnh nhân trong khoảng từ 0 đến 10 |


**Name:** Boston housing Dataset

| Variable | Definition                                                                |
|----------|---------------------------------------------------------------------------|
| crim     | Tỉ lệ tội phạm bình quân đầu người theo thị trấn                          |
| zn       | Tỉ lệ đất ở được quy hoạch cho các lô trên 25.000 sq.ft                   |
| indus    | Tỉ lệ diện tích kinh doanh phi bán lẻ trên mỗi thị trấn                   |
| chas     | Biến giả sông Charles (1 nếu đường bao quanh sông Charles; 0 nếu không)   |
| nox      | Nồng độ oxit nitric (phần trên 10 triệu)                                  |
| rm       | Số phòng bình quân mỗi hộ                                                 |
| age      | Tỷ lệ các đơn vị đã được sở hữu được xây dựng trước năm 1940              |
| dis      | Khoảng cách trọng số đến năm trung tâm việc làm ở Boston                  |
| rad      | Chỉ số tiếp cận các đường cao tốc hướng tâm                               |
| tax      | Thuế bất động sản trên 10.000 đô la                                       |
| ptratio  | Tỉ lệ học sinh-giáo viên theo thị trấn                                    |
| b        | 1000(Bk - 0.63)^2, trong đó Bk là tỷ lệ dân số da đen                     |
| lstat    | % dân số địa vị thấp                                                      |
| medv     | Giá trị trung bình của những ngôi nhà có chủ sở hữu ở trong khoảng $1000  |


In [2]:
prostate = h2o.upload_file("./Prostate Dataset/prostate.csv")
boston = h2o.upload_file("./Boston Housing dataset/BostonHousing.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


- With Prosates dataset

In [3]:
# convert columns to factors
prostate['CAPSULE'] = prostate['CAPSULE'].asfactor()
prostate['RACE'] = prostate['RACE'].asfactor()
prostate['DCAPS'] = prostate['DCAPS'].asfactor()
prostate['DPROS'] = prostate['DPROS'].asfactor()

# set the predictor and response columns
predictors_prostate = ["AGE", "RACE", "VOL", "GLEASON"]
response_col = "CAPSULE"

- With Boston Housing dataset

In [4]:
# set the predictor columns
predictors_boston = boston.columns[:-1]

# this example will predict the medv column
# set the response column to "medv", which is the median value of owner-occupied homes in $1000's
response = "medv"

# convert the `chas` column to a factor
# `chas` = Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
boston['chas'] = boston['chas'].asfactor()

### b. Model demo

### b.1. Generalized Linear Estimator Model

- With Prosates dataset

**Split dataset and build model**

In [5]:
# split into train and testing sets
train, test = prostate.split_frame(ratios = [0.8], seed = 1234)

# set GLM modeling parameters
# and initialize model training
glm_model = H2OGeneralizedLinearEstimator(family= "binomial",
                                          lambda_ = 0,
                                          compute_p_values = True)
glm_model.train(predictors_prostate, response_col, training_frame= prostate)

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_model_python_1679500168839_1


GLM Model: summary
    family    link    regularization    number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  ------  ----------------  ----------------------------  -----------------------------  ----------------------  ----------------
    binomial  logit   None              5                             5                              4                       py_4_sid_a113

ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.18215831942350072
RMSE: 0.4268000930453281
LogLoss: 0.5406879877388552
AUC: 0.7857533615502001
AUCPR: 0.722375044022577
Gini: 0.5715067231004003
Null degrees of freedom: 379
Residual degrees of freedom: 374
Null deviance: 512.2888401848891
Residual deviance: 410.9228706815299
AIC: 422.9228706815299

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3263180019526919
       0    1    Error    Rate
-----  ---  ---  -------  -------------
0      148  79   0.348    (79.0/227.0)
1      32   121  0.2092   (32.0/153.0)
Total  180  200  0.2921   (111.0/380.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.326318     0.685552  143
max f2                       0.19966      0.803279  230
max f0point5                 0.546868     0.673813  86
max accuracy                 0.47701      0.736842  120
max precision                0.96424      1         0
max recall                   0.0819482    1         282
max specificity              0.96424      1         0
max absolute_mcc             0.47701      0.456656  120
max min_per_class_accuracy   0.370958     0.713656  134
max mean_per_class_accuracy  0.47701      0.729665  120
max tns                      0.96424      227       0
max fns                      0.96424      152       0
max fps                      0.000253294  227       302
max tps                      0.0819482    153       282
max tnr                      0.96424      1         0
max fnr                      0.96424      0.993464  0
max fpr                      0.000253294  1         302
max tpr                      0.0819482    1         282

Gains/Lift Table: Avg response rate: 40.26 %, avg score: 28.72 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0105263                   1.02268            1.24183   1.24183            0.5              1.26458   0.5                         1.26458             0.0130719       0.0130719                  24.183    24.183             0.00426132
2        0.0210526                   0.636497           0         0.620915           0                0.778824  0.25                        1.0217              0               0.0130719                  -100      -37.9085           -0.0133598
3        0.0315789                   0.568999           1.24183   0.827887           0.5              0.598727  0.333333                    0.880709            0.0130719       0.0261438                  24.183    -17.2113           -0.0090985
4        0.0421053                   0.5313             1.24183   0.931373           0.5              0.5427    0.375                       0.796207            0.0130719       0.0392157                  24.183    -6.86275           -0.00483718
5        0.05                        0.496304           0.827887  0.915033     

**Predict on test dataset**

In [6]:
# predict using the model and the testing dataset
predict = glm_model.predict(test)

# View a summary of the prediction
predict.head()

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict,p0,p1,StdErr
1,0.531899,0.468101,0.447235
0,0.72698,0.27302,0.441374
0,0.900948,0.0990524,0.352853
0,0.906294,0.0937063,0.287441
1,0.641476,0.358524,0.271974
1,0.509269,0.490731,0.400724
1,0.355024,0.644976,0.235607
1,0.304671,0.695329,1.33002
1,0.472833,0.527167,0.170934
0,0.720066,0.279934,0.221276


- With Boston Housing dataset

**Split dataset and build model**

In [7]:

# split into train and testing sets
train, test = boston.split_frame(ratios = [0.8], seed = 1234)

# set the `alpha` parameter to 0.25
# then initialize the estimator then train the model
boston_glm = H2OGeneralizedLinearEstimator(alpha = 0.25)
boston_glm.train(x = predictors_boston,
                 y = response,
                 training_frame = train)

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_model_python_1679500168839_3


GLM Model: summary
    family    link      regularization                                number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  --------  --------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------
    gaussian  identity  Elastic Net (alpha = 0.25, lambda = 0.0269 )  14                            14                             1                       py_11_sid_a113

ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 19.963895377031225
RMSE: 4.468097512032523
MAE: 3.1489972498311634
RMSLE: NaN
Mean Residual Deviance: 19.963895377031225
R^2: 0.7601586334760372
Null degrees of freedom: 413
Residual degrees of freedom: 399
Null deviance: 34460.49697713591
Residual deviance: 8265.052686090927
AIC: 2446.366225652497

Scoring History: 
    timestamp            duration    iterations    negative_log_likelihood    objective          training_rmse      training_deviance    training_mae        training_r2
--  -------------------  ----------  ------------  -------------------------  -----------------  -----------------  -------------------  ------------------  ------------------
    2023-03-22 22:49:51  0.000 sec   0             34460.496932367154         83.23791529557283
    2023-03-22 22:49:51  0.025 sec   1                                                           4.468097512032523  19.963895377031225   3.1489972498311634  0.7601586334760372

Variable Importances: 
variable    relative_importance    scaled_importance    percentage
----------  ---------------------  -------------------  ------------
lstat       3.41617                1                    0.155628
dis         2.74048                0.802208             0.124846
rm          2.69599                0.789186             0.122819
ptratio     2.12732                0.622722             0.0969128
rad         1.83224                0.536344             0.0834699
nox         1.58398                0.463671             0.07216
tax         1.5166                 0.443948             0.0690906
chas.0      1.5011                 0.439412             0.0683846
chas.1      1.5011                 0.439412             0.0683846
zn          1.05511                0.308858             0.0480669
b           0.850247               0.248889             0.038734
crim        0.849731               0.248738             0.0387105
indus       0.162368               0.0475294            0.00739688
age         0.118465               0.0346777            0.00539681

[tips]
Use `model.explain()` to inspect the model.
--
Use `h2o.display.toggle_user_tips()` to switch on/off this section.

**Predict test dataset**

In [8]:
# predict using the model and the testing dataset
predict = boston_glm.predict(test)

# View a summary of the prediction
predict.head()

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict
28.2943
19.4569
19.0823
16.9093
16.2314
18.2361
12.6945
17.5583
15.4797
20.7294


## 2.2 On big dataset

### a. Dataset description

**Name**: Black Friday Dataset

**Size**: test : 9374 KB, train : 24.928 KB

| Variable                   | Description                                          |
|----------------------------|------------------------------------------------------|
| User_ID                    | User ID                                              |
| Product_ID                 | Product ID                                           |
| Gender                     | Sex of User                                          |
| Age                        | Age in bins                                          |
| Occupation                 | Occupation (Masked)                                  |
| City_Category              | Category of the City (A,B,C)                         |
| Stay_In_Current_City_Years | Number of years stay in current city                 |
| Marital_Status             | Marital Status                                       |
| Product_Category_1         | Product Category (Masked)                            |
| Product_Category_2         | Product may belongs to other category also (Masked)  |
| Product_Category_3         | Product may belongs to other category also (Masked)  |

In [29]:
test = "./Black friday dataset/test.csv"
train = "./Black friday dataset/train.csv"
test = h2o.upload_file(test)
train = h2o.upload_file(train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


### b. Model demo

## Supervised learning model

### b.1. Generalized Linear Models (GLM)

GLM is a linear regression technique that can be used for both binary and multiclass classification tasks. It is a popular algorithm for modeling categorical response variables.

**Split the data into train and validation sets**

In [30]:
train, validation = train.split_frame(ratios=[.8], seed=1234)

**Convert to dataframe**

In [31]:
test_df = test.as_data_frame()
train_df = train.as_data_frame()

In [12]:
print(f"Columns: {train.col_names}")
print("Train data shape: ", train_df.shape)
print("Test data shape: ", test_df.shape)

Columns: ['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3', 'Purchase']
Train data shape:  (440351, 12)
Test data shape:  (233599, 11)


In [32]:
predictors= ['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']
response_col = 'Purchase'

**Build a GLM model**

In [14]:
# Generate a GLM model using the training dataset
glm_model = H2OGeneralizedLinearEstimator(family = "gaussian", model_id = "glm_model", seed = 1234)
glm_model.train(x = predictors, y = response_col, training_frame = train, validation_frame = validation)

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: glm_model


GLM Model: summary
    family    link      regularization                               number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  --------  -------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------
    gaussian  identity  Elastic Net (alpha = 0.5, lambda = 3.4422 )  18                            17                             1                       py_15_sid_a113

ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 23205568.056807783
RMSE: 4817.215799277398
MAE: 3849.436524641251
RMSLE: 0.7343726988378344
Mean Residual Deviance: 23205568.056807783
R^2: 0.0792042006506739
Null degrees of freedom: 440350
Residual degrees of freedom: 440333
Null deviance: 11097569196779.87
Residual deviance: 10218595099383.363
AIC: 8718010.166484067

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 23321662.078232635
RMSE: 4829.25067461119
MAE: 3863.76604824862
RMSLE: 0.7379020766494636
Mean Residual Deviance: 23321662.078232635
R^2: 0.07999514384675566
Null degrees of freedom: 109716
Residual degrees of freedom: 109699
Null deviance: 2781274212318.6704
Residual deviance: 2558782798237.45
AIC: 2172738.5441739997

Scoring History: 
    timestamp            duration    iterations    negative_log_likelihood    objective           training_rmse      training_deviance    training_mae       training_r2         validation_rmse    validation_deviance    validation_mae    validation_r2
--  -------------------  ----------  ------------  -------------------------  ------------------  -----------------  -------------------  -----------------  ------------------  -----------------  ---------------------  ----------------  -------------------
    2023-03-22 22:50:03  0.000 sec   0             11097569196779.91          25201644.135655217
    2023-03-22 22:50:03  0.802 sec   1                                                            4817.215799277398  23205568.056807783   3849.436524641251  0.0792042006506739  4829.25067461119   23321662.078232635     3863.76604824862  0.07999514384675566

Variable Importances: 
variable                    relative_importance    scaled_importance    percentage
--------------------------  ---------------------  -------------------  ------------
Product_Category_1          604.696                1                    0.516414
Product_Category_2          232.835                0.385045             0.198842
City_Category.C             66.8927                0.110622             0.0571267
Gender.F                    53.6387                0.0887036            0.0458078
Gender.M                    53.6387                0.0887036            0.0458078
City_Category.A             43.7118                0.0722871            0.0373301
Occupation                  32.5519                0.0538319            0.0277996
City_Category.B             22.1809                0.0366811            0.0189427
Age.18-25                   12.2523                0.0202618            0.0104635
Age.51-55                   11.8639                0.0196196            0.0101318
Product_Category_3          11.2175                0.0185507            0.00957983
Age.36-45                   6.6539                 0.0110037            0.00568247
Stay_In_Current_City_Years  6.57539                0.0108739            0.00561542
Age.0-17                    5.04489                0.00834284           0.00430836
Age.26-35                   3.30628                0.00546767           0.00282358
Age.55+                     2.34441                0.003877             0.00200214
Marital_Status              1.54758                0.00255927           0.00132164
Age.46-50                   0                      0                    0

[tips]
Use `model.explain()` to 

**Predict on the test set**

In [15]:
glm_predict = glm_model.predict(test)
glm_predict.head()

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict
9892.85
9979.38
9006.67
9389.26
9643.42
10177.5
9964.98
10115.4
8392.54
9140.22


b.2. Gradient Boosting Machine (GBM)

H2O-3's Gradient Boosting Machines (GBM) is a high-performance implementation of the GBM algorithm designed for large-scale, distributed datasets. It is part of the H2O-3 machine learning platform, which is built on top of Apache Hadoop and Apache Spark, and can be used with R, Python, and Java.

H2O-3's GBM builds an ensemble of decision trees using a gradient descent algorithm to minimize a loss function. The algorithm starts by building a single decision tree and then iteratively improves the model by adding more trees to the ensemble, with each tree built to correct the errors of the previous tree. During training, the algorithm also uses a technique called bagging to sample the data and reduce overfitting.

One of the strengths of H2O-3's GBM is its ability to handle large datasets. It can take advantage of distributed computing frameworks like Hadoop and Spark to scale to hundreds or thousands of nodes, making it well-suited for big data applications. Additionally, it includes several advanced features such as early stopping, grid search, and cross-validation, which can help optimize the model and prevent overfitting.

**Build a GBM model**

In [16]:
#generate GBM model 
from h2o.estimators.gbm import H2OGradientBoostingEstimator
gbm_model = H2OGradientBoostingEstimator(model_id = "gbm_model", seed = 1234)
gbm_model.train(x = predictors, y = response_col, training_frame = train, validation_frame = validation)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,50.0,50.0,22861.0,5.0,5.0,5.0,29.0,32.0,31.84
,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2023-03-22 22:50:08,0.792 sec,0.0,5020.1239164,4066.6085068,25201644.1356552,5034.8315209,4082.2996568,25349528.4442582
,2023-03-22 22:50:09,2.280 sec,1.0,4735.0774868,3801.3252819,22420958.8062674,4750.0093100,3816.0798646,22562588.4449465
,2023-03-22 22:50:10,3.273 sec,2.0,4490.8967557,3571.0716210,20168153.6707122,4505.9871410,3584.9315484,20303920.1152356
,2023-03-22 22:50:11,3.617 sec,3.0,4282.9666143,3382.6486421,18343803.0194185,4298.1711983,3395.6240251,18474275.6501334
,2023-03-22 22:50:11,4.104 sec,4.0,4106.4302464,3218.4954281,16862769.3683754,4121.7116050,3230.8068728,16988506.5545139
,2023-03-22 22:50:11,4.461 sec,5.0,3956.1323769,3075.1757136,15650983.3833612,3971.1853697,3086.4646873,15770313.2406981
,2023-03-22 22:50:15,8.535 sec,29.0,2989.4930811,2275.6388124,8937068.8818987,3000.6901047,2283.4465542,9004141.1043078
,2023-03-22 22:50:19,11.803 sec,50.0,2947.2097562,2227.5339893,8686045.3470110,2960.5819133,2236.2321218,8765045.2653128


In [17]:
#predict using GBM model
test['Purchase'] = gbm_model.predict(test)
test['Purchase']

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


Purchase
13243.4
10452.9
6129.75
2708.07
2834.88
11730
13546.4
11351.8
17538.4
6171.97


### b.3. Deep learning with Neural Network

H2O-3's deeplearning model is a type of neural network designed for large-scale, distributed datasets. It is part of the H2O-3 machine learning platform, which is built on top of Apache Hadoop and Apache Spark, and can be used with R, Python, and Java.

The deeplearning model in H2O-3 is a fully connected, feedforward neural network that can be used for supervised learning tasks such as classification and regression. It consists of multiple layers of neurons, each of which performs a nonlinear transformation on the input data. During training, the network adjusts the weights of these neurons to minimize a loss function using backpropagation.

One of the strengths of H2O-3's deeplearning model is its ability to handle large datasets. It can take advantage of distributed computing frameworks like Hadoop and Spark to scale to hundreds or thousands of nodes, making it well-suited for big data applications. Additionally, it includes several advanced features such as dropout, L1/L2 regularization, and adaptive learning rates, which can help optimize the model and prevent overfitting.

**Build a Neural Network model**

In [18]:
dl_model = H2ODeepLearningEstimator(model_id = "dl_model", seed = 1234)
dl_model.train(x = predictors, y = response_col, training_frame = train, validation_frame = validation)

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: dl_model


Status of Neuron Layers: predicting Purchase, regression, gaussian distribution, Quadratic loss, 44,801 weights/biases, 534.8 KB, 4,501,205 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms              momentum    mean_weight           weight_rms           mean_bias            bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  --------------------  ----------  --------------------  -------------------  -------------------  -----------------------
    1        21       Input      0.0
    2        200      Rectifier  0.0        0.0   0.0   0.15051177353627238   0.3617798089981079    0.0         -0.08070053734497379  0.2514827251434326   -0.3344146904079263  0.7119884490966797
    3        200      Rectifier  0.0        0.0   0.0   0.3383351168232286    0.2484375238418579    0.0         -0.06116506109575535  0.20994144678115845  -0.1353003100729424  1.0130534172058105
    4        1        Linear                0.0   0.0   0.007616981626852066  0.004353361204266548  0.0         0.005263040255795204  0.17538803815841675  -1.697200292168398   1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 8586622.342219016
RMSE: 2930.2939003142696
MAE: 2217.7875744312405
RMSLE: NaN
Mean Residual Deviance: 8586622.342219016

ModelMetricsRegression: deeplearning
** Reported on validation data. **

MSE: 8922741.959246764
RMSE: 2987.0959072729424
MAE: 2250.5799697623743
RMSLE: NaN
Mean Residual Deviance: 8922741.959246764

Scoring History: 
    timestamp            duration          training_speed    epochs    iterations    samples      training_rmse    training_deviance    training_mae    training_r2    validation_rmse    validation_deviance    validation_mae    validation_r2
--  -------------------  ----------------  ----------------  --------  ------------  -----------  ---------------  -------------------  --------------  -------------  -----------------  ---------------------  ----------------  ---------------
    2023-03-22 22:50:24  0.000 sec                           0         0             0            nan              nan                  nan             nan            nan                nan                    nan               nan
    2023-03-22 22:50:32  12.149 sec        15074 obs/sec     0.227073  1             99992        3621.72          1.31169e+07          2636.92         0.475764       3677.73            1.35257e+07            2673.12           0.466432
    2023-03-22 22:51:02  41.968 sec        38340 obs/sec     2.95408   13            1.30083e+06  2959.25          8.75717e+06          2215.06         0.650006       3009.09            9.05461e+06            2242.33           0.642809
    2023-03-22 22:51:28  1 min  8.060 sec  45077 obs/sec     5.90431   26            2.59997e+06  2934.68          8.61234e+06          2210.2          0.655794       3001.23            9.00739e+06            2249.94           0.644672
    2023-03-22 22:51:52  1 min 31.290 sec  48189 obs/sec     8.63168   38            3.80097e+06  2930.29          8.58662e+06          2217.79         0.656822       2987.1             8.92274e+06            2250.58           0.648011
    2023-03-22 22:52:07  1 min 46.129 sec  49140 obs/sec     10.2219   45            4.5012e+06   2931.13          8.59155e+06          2181.45         0.656625       2994.09            8.96458e+06            2216.79           0.646361
    2023-03-22 22:52:09  1 min 48.223 sec  49133 obs/sec     10.2219   45            4.5012e+06   2930.29          8.58662e+06          2217.79         0.656822       2987.1             8.92274e+06            2250.58           0.648011

Variable Importances: 
variable                    relative_importance    scaled_importance    percentage
--------------------------  ---------------------  ------------------- 

**Predict on the test set**

In [19]:
#predict using Neural Network model
test['Purchase'] = dl_model.predict(test)
test['Purchase']

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


Purchase
14003.5
10456.7
5619.17
1740.58
2475.64
11750.7
12733.2
10498.6
18645.1
5941.12


### b.5. Miscellaneous models RuleFit


RuleFit is a supervised learning algorithm that combines the interpretability of decision rules with the accuracy of ensembles of trees.

**Build a RuleFit model**

In [20]:
rulefit_model = H2ORuleFitEstimator(model_id = "rulefit_model", seed = 1234)
rulefit_model.train(x = predictors, y = response_col, training_frame = train, validation_frame = validation)

rulefit Model Build progress: |██████████████████████████████████████████████████| (done) 100%


,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,rule_ensemble_size,number_of_trees,number_of_internal_trees,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,gaussian,identity,Lasso (lambda = 1.7211 ),353,116,2,335.0,50.0,50.0,0.0,3.0,3.0,0.0,8.0,6.7


**Predict on the test set**

In [21]:
rulefit_predict = rulefit_model.predict(test)
rulefit_predict.head()

rulefit prediction progress: |███████████████████████████████████████████████████| (done) 100%


predict
13133.5
9857.8
6149.09
5933.77
2524.27
11985.7
13786.9
10787.2
7563.92
5949.67


### b.5. AutoML 

H2O-3's AutoML (Automatic Machine Learning) models are a type of machine learning algorithm that automates the entire process of building a machine learning model.

AutoML models automatically perform tasks such as data preprocessing, feature engineering, algorithm selection, hyperparameter tuning, and model ensembling. This allows non-experts to build powerful machine learning models without requiring extensive knowledge of the underlying algorithms or programming languages.

H2O-3's AutoML models use a range of algorithms such as GLM, GBM, XGBoost, Deep Learning, and Stacked Ensembles to create models that are optimized for the specific problem at hand. The algorithm selection and hyperparameter tuning are done through a process of trial and error, where the model performance is evaluated on a validation dataset and the best-performing models are selected.

AutoML models are particularly useful when working with large datasets or when there is a need to quickly build a high-performing machine learning model. They can be used for both regression and classification tasks, and are able to handle a wide range of data types and formats.

**Build an AutoML model**

H2O-3 AutoML can be used to build a variety of models, including GBMs, GLMs, and deep learning models. AutoML can also be used to build stacked ensembles of the models it builds. In this example, we will build 10 models to compare their performance.

In [33]:
#generate AutoML model
aml = H2OAutoML(max_models = 10, seed = 1)
aml.train(x = predictors, y = response_col, training_frame = train, validation_frame = validation)

AutoML progress: |
23:12:02.490: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
23:12:02.516: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),4/10
# GBM base models (used / total),3/6
# DeepLearning base models (used / total),0/1
# DRF base models (used / total),1/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


**Predict on the test set**

In [34]:
#predict using AutoML model
aml_predict = aml.predict(test)
aml_predict.head()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict
14635.9
10627.9
6526.35
2531.58
2550.15
11292.3
13089
10909.2
18122.7
6005.26


H2O-3 AutoML has built a total of 10 models. Let’s look at the leaderboard to see the performance of each model.

In [36]:
leaderboard = aml.leaderboard
leaderboard.head(rows=leaderboard.nrows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20230322_231202,2867.89,8.22482e+06,2141.37,nan,8.22482e+06
StackedEnsemble_BestOfFamily_1_AutoML_1_20230322_231202,2870.06,8.23726e+06,2140.52,nan,8.23726e+06
GBM_1_AutoML_1_20230322_231202,2870.71,8.24095e+06,2140.85,nan,8.24095e+06
GBM_4_AutoML_1_20230322_231202,2877.08,8.27761e+06,2152.4,nan,8.27761e+06
GBM_3_AutoML_1_20230322_231202,2885.44,8.32577e+06,2162.06,nan,8.32577e+06
GBM_2_AutoML_1_20230322_231202,2903.32,8.42926e+06,2180.61,nan,8.42926e+06
GBM_5_AutoML_1_20230322_231202,2915.81,8.50197e+06,2192.54,nan,8.50197e+06
GBM_grid_1_AutoML_1_20230322_231202_model_1,2929.58,8.58247e+06,2209.29,nan,8.58247e+06
DeepLearning_1_AutoML_1_20230322_231202,3049.96,9.30224e+06,2310.87,nan,9.30224e+06
DRF_1_AutoML_1_20230322_231202,3283.24,1.07797e+07,2461.05,0.551358,1.07797e+07


In [ ]:
best_model = aml.leader
best_model

key,value
Stacking strategy,cross_validation
Number of base models (used / total),4/10
# GBM base models (used / total),3/6
# DeepLearning base models (used / total),0/1
# DRF base models (used / total),1/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [ ]:
h2o.cluster().shutdown()

H2O session _sid_a113 closed.
